In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import matplotlib.ticker as ticker
import urllib
import sys
import os
import zipfile
#import glove

import spacy

from math import sqrt
from statistics import stdev
import csv
from numpy import dot
from numpy.linalg import norm

import comparison_functions
import effect_sizes

In [2]:
import spacy
print(spacy.__version__)  # spaCy version
print(spacy.__file__)

2.1.3
/Users/karie.moorman/anaconda/lib/python3.6/site-packages/spacy/__init__.py


In [5]:
#loading word vector models into spaCy format: example below = glove.6b.300d.txt

from __future__ import unicode_literals
import numpy
import plac
import spacy
import tqdm

#@plac.annotations(
#  vectors_loc=("location of input word vector model", "positional", None, str),
#  out_loc=("location of output word vector model in spacy format", "positional", None, str),
#  lang=("en", "positional", None, str),
#)

def main(vectors_loc, out_loc, lang=None):
  if lang is None:
    lang = 'en'
  nlp = spacy.blank(lang)
  print('Loading GloVe vectors: {}'.format(vectors_loc))
  with open(vectors_loc, 'r') as file_:
    lines = file_.readlines()
    print('Assigning {:,} spaCy vectors'.format(len(lines)))
    for line in tqdm.tqdm(lines, leave=False):
      pieces = line.split(' ')
      word = pieces[0]
      vector = numpy.asarray([float(v) for v in pieces[1:]], dtype='f')
      nlp.vocab.set_vector(word, vector)
  print('Saving spaCy vector model: {}'.format(out_loc))
  nlp.to_disk(out_loc)
  print('Done.')

main("models/glove.6B.300d.txt", "models/glove6B300d", 'en')



Loading GloVe vectors: models/glove.6B.300d.txt


  0%|          | 229/400000 [00:00<02:55, 2278.38it/s]

Assigning 400,000 spaCy vectors


Saving spaCy vector model: models/glove6B300d
Done.


In [7]:
nlp2 = spacy.load("models/glove6B300d")

In [9]:
word2 = nlp2.vocab[u'with']

comparison_functions.similar_strings(nlp2,word2)

Top 20 most similar words to with :


['all',
 'that',
 'so',
 'they',
 'when',
 'there',
 'are',
 'not',
 'the',
 'a',
 'good',
 'it',
 'have',
 'had',
 'were',
 'is',
 'has',
 'any',
 'he',
 'would']

In [12]:
tokens = nlp2(u'dog cat banana afskfsd')

#commonality of word 
for token in tokens:
    print(token.text, token.has_vector, token.vector_norm, token.is_oov)

dog True 6.673079 True
cat True 6.326409 True
banana True 6.560879 True
afskfsd False 0.0 True


In [ ]:
tokens = nlp2(u'dog cat banana apple')

#word similarity
for token1 in tokens:
    for token2 in tokens:
        print(token1.text, token2.text, token1.similarity(token2))

In [ ]:
doc1 = nlp2(u"illegals undocumented refugees immigrants foreigners")
doc2 = nlp2(u"sneaky deceitful mean rude careless")
Citizen = nlp2(u'citizens inhabitants locals natives residents')
Good = nlp2(u'sincere honest trustworthy share nice')
print("Noncitizen & Good:", doc1.similarity(Good))
print("Noncitizen & Bad:", doc1.similarity(doc2))
print("Citizen & Good:", Citizen.similarity(Good))
print("Citizen & Bad:", Citizen.similarity(doc2))


In [ ]:
Noncitizen = (u'illegals undocumented refugees immigrants foreigners') #list of strings = input at present
Citizen = (u'citizens inhabitants locals natives residents')
Good = (u'sincere honest trustworthy share nice')
Bad = (u'sneaky deceitful mean rude careless')

In [ ]:
results_citizen_good2 = comparison_functions.compare_wordlists_by_spacy_vectors(nlp2, Citizen, Good)
results_citizen_bad2 = comparison_functions.compare_wordlists_by_spacy_vectors(nlp2, Citizen, Bad)

results_noncitizen_good2 = comparison_functions.compare_wordlists_by_spacy_vectors(nlp2, Noncitizen, Good)
results_noncitizen_bad2 = comparison_functions.compare_wordlists_by_spacy_vectors(nlp2, Noncitizen, Bad)

results_within_citizen2 = comparison_functions.compare_wordlists_by_spacy_vectors(nlp2, Citizen)
results_within_noncitizen2 = comparison_functions.compare_wordlists_by_spacy_vectors(nlp2, Noncitizen)

In [ ]:
df = pd.DataFrame(results_citizen_good2)
df

In [ ]:
cohens_d_Citizen_Noncitizen2 = effect_sizes.cohens_d(results_citizen_good2, results_citizen_bad2, results_noncitizen_good2, results_noncitizen_bad2)

In [ ]:
#word vector similarities using spaCy models

parser = spacy.load('en_core_web_lg')
word = parser.vocab[u'illegals']

comparison_functions.Similar_strings(parser,word)

In [ ]:
#word pair similarities using spaCy models

nlp = spacy.load('en_core_web_lg')
Noncitizen = (u'illegals undocumented refugees immigrants foreigners') #list of strings = input at present
Citizen = (u'citizens inhabitants locals natives residents')
Good = (u'sincere honest trustworthy share nice')
Bad = (u'sneaky deceitful mean rude careless')


In [ ]:
results_citizen_good = comparison_functions.compare_wordlists_by_spacy_vectors(nlp, Citizen, Good)
results_citizen_bad = comparison_functions.compare_wordlists_by_spacy_vectors(nlp, Citizen, Bad)

results_noncitizen_good = comparison_functions.compare_wordlists_by_spacy_vectors(nlp, Noncitizen, Good)
results_noncitizen_bad = comparison_functions.compare_wordlists_by_spacy_vectors(nlp, Noncitizen, Bad)

results_within_citizen = comparison_functions.compare_within_wordlist_by_spacy_vectors(nlp, Citizen)
results_within_noncitizen = comparison_functions.compare_within_wordlist_by_spacy_vectors(nlp, Noncitizen)

In [ ]:
df = pd.DataFrame(results_citizen_good)
df

In [ ]:
df.loc[:,2].mean()

In [ ]:
mean_diff_Citizen_Noncitizen = effect_sizes.diff_of_means(results_citizen_good, results_citizen_bad, results_noncitizen_good, results_noncitizen_bad)

In [ ]:
cohens_d_Citizen_Noncitizen = effect_sizes.cohens_d(results_citizen_good, results_citizen_bad, results_noncitizen_good, results_noncitizen_bad)

In [ ]:
##def load_glove_model(glove_file):
##    print("Loading Glove Model")
##    f = open(glove_file, 'r', errors = 'ignore', encoding='utf8')
##    model = {}
##    vector_size = 300
##    for line in f:
##        split_line = line.split()
##        word = " ".join(split_line[0:len(split_line) - vector_size])
##        embedding = np.array([float(val) for val in split_line[-vector_size:]])
##        model[word] = embedding
##    print("Done.\n" + str(len(model)) + " words loaded!")
##    return model
##
##
##glove = load_glove_model("models/glove.6B.300d.txt")

#works on models/glove.6B.300d.txt
#print(glove["hello", "goodbye"])


In [ ]:
##from gensim.scripts.glove2word2vec import glove2word2vec
##glove2word2vec(glove_input_file="models/glove.6B.300d.txt", word2vec_output_file="gensim_glove6B300d_vectors.txt")

In [ ]:
##from gensim.models.keyedvectors import KeyedVectors
##glove_model = KeyedVectors.load_word2vec_format("gensim_glove6B300d_vectors.txt", binary=False)

In [ ]:
##def diff_of_means(wordlist1, wordlist2, wordlist3, wordlist4):
##    df1 = pd.DataFrame(wordlist1)
##    df2 = pd.DataFrame(wordlist2)
##    df3 = pd.DataFrame(wordlist3)
##    df4 = pd.DataFrame(wordlist4)
##    a1 = df1.loc[:,2].mean()
##    a2 = df2.loc[:,2].mean()
##    b1 = df3.loc[:,2].mean()
##    b2 = df4.loc[:,2].mean()
##    d1= (np.add(a1,a2))/2
##    d2= (np.add(b1,b2))/2
##    d3= np.subtract(d1,d2)
##    print("Group 1:",a1, a2)
##    print("Group 2:",b1, b2)
##    print("The average similarity for group 1 is", d1)
##    print("The average similarity for group 2 is", d2)
##    print("The difference of the means is", d3)  
##
##def cohens_d (wordlist1, wordlist2, wordlist3, wordlist4):
##    df1 = pd.DataFrame(wordlist1)
##    df2 = pd.DataFrame(wordlist2)
##    df3 = pd.DataFrame(wordlist3)
##    df4 = pd.DataFrame(wordlist4)
##    a1 = df1.loc[:,2].mean()
##    a2 = df2.loc[:,2].mean()
##    a3 = pd.concat([df1,df2])
##    b1 = df3.loc[:,2].mean()
##    b2 = df4.loc[:,2].mean()
##    b3 = pd.concat([df3,df4])
##    # calculate the mean similarity of the samples
##    #d1= (np.add(a1,a2))/2
##    #d2= (np.add(b1,b2))/2
##    d4 = a3.loc[:,2].mean()
##    d5 = b3.loc[:,2].mean()
##    # calculate the difference of the means
##    d3= np.subtract(d4,d5)
##    # calculate the pooled standard deviation
##    s = sqrt((np.std(a3.loc[:,2]) ** 2 + np.std(b3.loc[:,2]) ** 2) / 2)
##    #calculate cohen's d
##    cohen_d = d3 / s
##    #return cohen_d
##    print("Group 1 similarity scores are", (a1, a2))
##    print("Group 2 similarity scores are", (b1, b2))
##    print("The global mean similarity score for Group 1 is", d4)
##    print("The global mean similarity score for Group 2 is", d5)
##    print("The difference of the means is", d3) 
##    #print(b3)
##    print("Cohen's D is", cohen_d)
##
##diff_of_means(results_citizen_good, results_citizen_bad, results_noncitizen_good, results_noncitizen_bad)    
##cohens_d(results_citizen_good, results_citizen_bad, results_noncitizen_good, results_noncitizen_bad)
    

In [13]:
nlp = spacy.load("en_core_web_lg")

In [14]:
text = u"""If discriminant validity is not established, then latent constructs are having an influence on the variation of more than just the observed variables to which they are theoretically related."""
doc=nlp(text)

In [15]:
for token in doc:
    print(token.text, token.pos_, token.dep_, token.shape_, token.is_alpha, token.is_stop)

If ADP mark Xx True True
discriminant ADJ compound xxxx True False
validity NOUN nsubjpass xxxx True False
is VERB auxpass xx True True
not ADV neg xxx True True
established VERB advcl xxxx True False
, PUNCT punct , False False
then ADV advmod xxxx True True
latent ADJ amod xxxx True False
constructs NOUN nsubj xxxx True False
are VERB aux xxx True True
having VERB ROOT xxxx True False
an DET det xx True True
influence NOUN dobj xxxx True False
on ADP prep xx True True
the DET det xxx True True
variation NOUN pobj xxxx True False
of ADP prep xx True True
more ADJ pobj xxxx True True
than ADP prep xxxx True True
just ADV advmod xxxx True True
the DET det xxx True True
observed ADJ amod xxxx True False
variables NOUN pobj xxxx True False
to PART prep xx True True
which DET pobj xxxx True True
they PRON nsubj xxxx True True
are VERB relcl xxx True True
theoretically ADV advmod xxxx True False
related ADJ acomp xxxx True False
. PUNCT punct . False False


In [16]:
from spacy import displacy



In [17]:
doc2 = nlp(u"Rats are various medium-sized, long-tailed rodents.")
displacy.render(doc2, style="dep", jupyter=True, options={'distance': 100})


In [18]:

doc2
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)


In [19]:
nlp2 = spacy.load("en_core_web_sm")

displacy.render(doc2, style="ent", jupyter=True)

/Users/karie.moorman/anaconda/lib/python3.6/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


In [21]:
from spacy import displacy
doc = nlp2(u"This is a sentence about Google.")
#displacy.serve(doc, style="dep") # run the web server
displacy.render(doc, style="ent", jupyter=True) # generate HTML

In [ ]:
#import sent2tree


#st = sent2tree.sentenceTree(doc)
#t, ts = st.render()
#t.render('%%inline', tree_style=ts)